In [ ]:
"""
succesfully_parsed = 0

parsed_pages = []
for (i, page) in enumerate(pages):
    try:
        parsed_pages.append(extract_info(page))
        succesfully_parsed += 1
    except Exception as e:
        print('Error parsing ' + page + ': ' + str(e))
    print(f"Parsed {i} out of {str(len(pages))}", end='\r')
print('Succesfully parsed ' + str(succesfully_parsed) + ' out of ' + str(len(pages)))
POSSIBLY USEFUL, FROM OLD SCRAPER"""

In [83]:
# Part 1.1
import requests
from bs4 import BeautifulSoup
import lxml # actually used in BS but vscode doesn't recognise it
import pandas as pd
import time
import os

In [84]:
pages = os.listdir("pages")
sitemap_url = "https://kurser.ku.dk/sitemap.xml" # the magic url

['AANA18106U.html', 'AANA18112U.html', 'AANA18123U.html', 'AANA18126U.html', 'AANA18127U.html', 'AANA18129U.html', 'AANA18131U.html', 'AANA18132U.html', 'AANA18133U.html', 'AANA18134U.html', 'AANA18136U.html', 'AANA20003U.html', 'AANA20004U.html', 'AANB05091U.html', 'AANB11015U.html', 'AANB11041U.html', 'AANB11073U.html', 'AANB18000U.html', 'AANB18001U.html', 'AANB18002U.html', 'AANB18003U.html', 'AANB18004U.html', 'AANB18005U.html', 'AANB18006U.html', 'AANB18007U.html', 'AANB18008U.html', 'AANB18009U.html', 'AANK13515U.html', 'AANK16004U.html', 'AANK16011U.html', 'AANK16012U.html', 'AANK16013U.html', 'APSB11015U.html', 'APSB11017U.html', 'APSB16101U.html', 'APSB21001U.html', 'APSB21002U.html', 'APSB21003U.html', 'APSB21004U.html', 'APSB21005U.html', 'APSB21006U.html', 'APSB21007U.html', 'APSB21008U.html', 'APSB21009U.html', 'APSB21010U.html', 'APSB21011U.html', 'APSB21012U.html', 'APSB21013U.html', 'APSB21014U.html', 'APSB21015U.html', 'APSB21016U.html', 'APSB21104U.html', 'APSB21710U

In [76]:
sitemap = requests.get(sitemap_url)
sitemap_soup = BeautifulSoup(sitemap.text, "lxml")
urls = [url.text for url in sitemap_soup.find_all("loc")][1:] # first is root
print("Number of urls:", len(urls))

def name_from_url(url):
    return url.split("/")[-1]

print("Example:", name_from_url(urls[0]))

/home/josh/.local/share/virtualenvs/disproject-K5ya_bLC/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


Number of urls: 3204
Example: HØEK0117EU


In [77]:
def get_page(url):
    name = name_from_url(url)
    try:
        with open(f"pages/{name}.html", "r") as f:
            return f.read()
    except FileNotFoundError:
        time.sleep(0.25) # be nice to the server
        page = requests.get(url)
        with open(f"pages/{name}.html", "w") as f:
            f.write(page.text)
        return page.text
    

In [78]:
# This step simply caches the pages so we don't have to download them again
for (i, url) in enumerate(urls):
    get_page(url)
    print(f"Got: {name_from_url(url)} Progress: {i+1}/{len(urls)}", end="\r")

In [79]:
def get_panel_info(url:str) -> dict:
    """
    This function attempts to grab:
    - Study board
    - Contracting department
    - Contracting faculty
    - Course coordinator
    -- Lecturers"""
    soup = BeautifulSoup(get_page(url), "html.parser")
    panel_bodies = soup.find_all("div", class_="panel-body")

    # Find the one with the most h5's, this is presumably the one we want
    panel_body = max(panel_bodies, key=lambda x: len(x.find_all("h5")))

    # Grabbing top elements
    dl = panel_body.find("dl", class_="dl-horizontal")
    top_elements = {dt.text: dd.text for dt, dd in zip(dl.find_all("dt"), dl.find_all("dd"))}


    # Grabbing bottom elements
    h5s = panel_body.find_all("h5")
    # For every h5, get the next div
    siblings = [h5.find_next_sibling() for h5 in h5s]

    # if a sibling is a ul, get the li's
    siblings = [s.find_all("li") if s.name == "ul" else s for s in siblings]

    # bottom elements are the 5(?) in the h5s
    def remove_spans(soup):
        """
        Removes all span tags from a soup, used to remove the mailto links
        """
        try:
            for span in soup.find_all("span"):
                span.decompose()
        except AttributeError:
            pass
        return soup

    bottom_elements = {h5.text: [remove_spans(sibling).text.strip() for sibling in siblings[i]] for i, h5 in enumerate(h5s)}
    final_dict = {
        "URL": url,
        **top_elements,
        **bottom_elements,
        "last-modified": panel_body.find("div", class_="last-modified").text}
    
    # Lowercase all keys
    return {k.lower(): v for k, v in final_dict.items()}
print(urls[240])
for key, value in get_panel_info(urls[1]).items():
    print(repr(key),":", repr(value))

http://kurser.ku.dk/course/HKUB10191U
'url' : 'http://kurser.ku.dk/course/HØEK0118EU'
'sprog' : 'Polsk - Delvist på dansk.'
'kursuskode' : 'HØEK0118EU'
'point' : '15 ECTS'
'niveau' : 'Kandidat'
'varighed' : '1 semester'
'placering' : 'Efterår'
'skemagruppe' : 'Se skemalink herunder'
'studienævn' : ['Studienævnet for Tværkulturelle og Regionale Studier']
'udbydende institut' : ['Institut for Tværkulturelle og regionale studier']
'udbydende fakultet' : ['Det Humanistiske Fakultet']
'kursusansvarlige' : ['Anita Pluwak']
'undervisere' : ['Anita Pluwak, pluwak@hum.ku.dk']
'last-modified' : 'Gemt den\n06-02-2023'


In [80]:
def get_course_items(url:str) -> dict:
    # Find a div with regex class * main-content
    soup = BeautifulSoup(get_page(url), "html.parser")
    main_content = soup.find("div", class_=lambda x: x and "main-content" in x)
    
    out_dict = {}
    out_dict["primary title"] = main_content.find("h1").text.strip()

    def english_title(soup):
        # If the english title exists, return else return None
        try:
            # it exists in a div with id course-language
            return soup.find("div", id="course-language").text.strip()
        except AttributeError:
            return None
        
    out_dict["english title"] = english_title(main_content)

    out_dict["course content"] = main_content.find("div", id="course-content").text.strip()

    out_dict["recommended prerequisites"] = main_content.find("div", id="course-skills").text.strip()

    def grab_exam_table(soup):
        exams = soup.find("div", id="course-exams1").find("dl")
        return {dt.text.strip(): dd.text.strip() for dt, dd in zip(exams.find_all("dt"), exams.find_all("dd"))}
    
    out_dict["exams"] =  grab_exam_table(main_content) 


    def grab_course_load(soup):
        course_load = soup.find("div", id="course-load")
        # li's are key-vals and should be zipped
        lis = course_load.find_all("li")
        lis = [li.text.strip() for li in lis]
        # zip every other element
        temp_dict = {k: v for k, v in zip(lis[::2], lis[1::2])}

        return temp_dict

        
    out_dict["course load"] = grab_course_load(main_content)

    return out_dict




for key, value in get_course_items("https://kurser.ku.dk/course/NMAB21015U").items():
    print(repr(key),":", repr(value))


'primary title' : 'NMAB21015U\xa0Sandsynlighedsteori\n(Sand)'
'english title' : 'Probability Theory (Sand)'
'course content' : 'Kurset giver en introduktion til sandsynlighedsteori i en\nmålteoretisk ramme. Der er speciel fokus på centrale resultater af\ngeneral betydning inden for matematik og statistik.KursusindholdDefinition af og egenskaber ved sandsynlighedsmålStokastiske variable; fordelinger af stokastiske variableTransformation af stokastiske variable og deres\nfordelingerTransformationssætninger for integralerUafhængighed; foldningKarakteristiske funktionerKonvergensbegreber for stokastiske variable; store tals\nlovDen flerdimensionale normalfordelingCentrale grænseværdisætning'
'recommended prerequisites' : 'Sandsynlighedsregning og\nstatistik (SS), om nødvendigt samtidigt.\n\nLebesgueintegralet og målteori (LIM) - alternativt Analyse 2 (An2)\nfra tidligere år.'
'exams' : {'Point': '7,5 ECTS', 'Prøveform': 'Skriftlig prøve, 4 timer med opsyn.', 'Krav til indstilling til eksam

In [81]:
def grab_all_info(url):
    return {
        **get_panel_info(url),
        **get_course_items(url)
    }

# test
for key, value in grab_all_info("https://kurser.ku.dk/course/NMAB21015U").items():
    print(repr(key),":", repr(value))

'url' : 'https://kurser.ku.dk/course/NMAB21015U'
'sprog' : 'Dansk'
'kursuskode' : 'NMAB21015U'
'point' : '7,5 ECTS'
'niveau' : 'Bachelor'
'varighed' : '1 blok'
'placering' : 'Blok 2'
'skemagruppe' : 'C'
'kursuskapacitet' : 'Ingen begrænsning\n\nDer kan være færre pladser i eftertilmeldingsperioden'
'studienævn' : ['Studienævn for Matematik og Datalogi']
'udbydende institut' : ['Institut for Matematiske Fag']
'udbydende fakultet' : ['Det Natur- og Biovidenskabelige Fakultet']
'kursusansvarlige' : ['Niels Richard Hansen']
'undervisere' : ['Niels Richard Hansen']
'last-modified' : 'Gemt den\n28-02-2023'
'primary title' : 'NMAB21015U\xa0Sandsynlighedsteori\n(Sand)'
'english title' : 'Probability Theory (Sand)'
'course content' : 'Kurset giver en introduktion til sandsynlighedsteori i en\nmålteoretisk ramme. Der er speciel fokus på centrale resultater af\ngeneral betydning inden for matematik og statistik.KursusindholdDefinition af og egenskaber ved sandsynlighedsmålStokastiske variable; fo